In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.metrics import r2_score

In [2]:
#!pip install lightgbm

In [3]:
df = pd.read_csv('Preprocessed_Podcast_data.csv')

In [4]:
df.columns

Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes', 'Genre',
       'Host_Popularity_percentage', 'Publication_Day', 'Publication_Time',
       'Guest_Popularity_percentage', 'Number_of_Ads', 'Episode_Sentiment',
       'Listening_Time_minutes'],
      dtype='object')

In [5]:
df=pd.get_dummies(df,columns=["Genre","Publication_Day","Publication_Time","Episode_Sentiment"],drop_first=True)

In [6]:
df.drop(["Podcast_Name","Episode_Title"],axis=1,inplace=True)

In [7]:
df.columns

Index(['Episode_Length_minutes', 'Host_Popularity_percentage',
       'Guest_Popularity_percentage', 'Number_of_Ads',
       'Listening_Time_minutes', 'Genre_Comedy', 'Genre_Education',
       'Genre_Health', 'Genre_Lifestyle', 'Genre_Music', 'Genre_News',
       'Genre_Sports', 'Genre_Technology', 'Genre_True Crime',
       'Publication_Day_Monday', 'Publication_Day_Saturday',
       'Publication_Day_Sunday', 'Publication_Day_Thursday',
       'Publication_Day_Tuesday', 'Publication_Day_Wednesday',
       'Publication_Time_Evening', 'Publication_Time_Morning',
       'Publication_Time_Night', 'Episode_Sentiment_Neutral',
       'Episode_Sentiment_Positive'],
      dtype='object')

In [8]:
indep_X=df.drop(['Listening_Time_minutes'], axis=1)

In [9]:
dep_Y = df['Listening_Time_minutes']

In [10]:
 test = SelectKBest(score_func=f_regression,k=2)
fit1= test.fit(indep_X,dep_Y)
selectk_features = fit1.transform(indep_X)
 # Get the mask of selected features (True means selected)
selected_mask = fit1.get_support()

# Get the names of selected features (using the mask)
selected_columns = indep_X.columns[selected_mask]

# Print selected features
print(f"Selected Features: {selected_columns}")

Selected Features: Index(['Episode_Length_minutes', 'Number_of_Ads'], dtype='object')


In [11]:
indep_X_kbest = df[selected_columns]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(indep_X_kbest, dep_Y, test_size=0.25, random_state=0)

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
rf = RandomForestRegressor(n_estimators=150, max_depth=20, min_samples_split=5, random_state=0)
rf.fit(X_train_scaled, y_train)



C:\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=150,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [15]:
y_pred_rf = rf.predict(X_test_scaled)
r2_rf = r2_score(y_test, y_pred_rf)
r2_rf

C:\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


0.8479837290777799

In [16]:
import pickle

In [17]:
filename="finalized_model_Podcast.sav"

In [18]:
pickle.dump(rf,open(filename,'wb'))

In [19]:
preinput=scaler.transform([[45,2]])  # episode_length, no.of.Ads

In [20]:
preinput

array([[-0.59556927,  0.61169271]])

In [21]:
loaded_model=pickle.load(open("finalized_model_Podcast.sav",'rb')) 

In [22]:
result=loaded_model.predict(preinput) 
result

C:\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


array([27.25286374])